In [1]:
!python --version

Python 3.10.6


In [2]:
import gym
import numpy as np
import random
import tensorflow as tf
from dataclasses import dataclass
import time

In [3]:
# !git clone https://github.com/yourusername/yourrepository.git
!git clone https://github.com/geomuse/snake.git
%cd snake

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'snake'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 13), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (25/25), 9.17 KiB | 4.59 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/snake


In [4]:
!ls
from gym_snake_inherit import SnakeEnv

advance_snake_gym_framework.py	gym_snake.py  united_snake.py
advance_snake.py		README.md
gym_snake_inherit.py		snake.py


In [5]:
'''
env = SnakeEnv()
observation = env.reset()
done = False

while not done:
    observation, reward, done, _ = env.sample()
'''
print('.')

.


In [ ]:
env = SnakeEnv()
observation = env.reset()
done = False

# Q-learning算法
def q_learning(env, num_episodes=1000, learning_rate=0.8, discount_factor=0.95, exploration_prob=0.2):
    # 初始化Q值表格，记录每个状态和动作的Q值
    num_actions = env.action_space.n
    num_states = env.observation_space.shape[0]
    q_table = np.zeros((num_states, num_actions))

    for episode in range(num_episodes):
        state = env.reset()
        done = False

        while not done:
            # 通过epsilon-greedy策略选择动作
            if random.uniform(0, 1) < exploration_prob:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            next_state, reward, done, _ = env.step(action)

            # 更新Q值函数
            target = reward
            if not done:
                target = reward + discount_factor * np.max(q_table[next_state, :])
            try :
              q_table[state, action] = (1 - learning_rate) * q_table[state, action] + learning_rate * target
            except :
              return q_table
            state = next_state

    return q_table

# 训练Q值函数
q_table = q_learning(env)

In [15]:
print(q_table)

[[-0.0494047  -0.02503034 -0.00226788 -0.06052233]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 ...
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
# 使用训练好的Q值函数进行游戏
import pygame
pygame.init()

state = env.reset()
done = False

while not done:
    action = np.argmax(q_table[state, :])
    next_state, _ , done, _ = env.step(action)
    env.render()
    state = next_state

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
from google.colab import files
import pickle

with open('q_table.pkl', 'wb') as f:
    pickle.dump(q_table, f)

files.download('q_table.pkl')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''
env = SnakeEnv()
observation = env.reset()
done = False

num_episodes = 1000
num_steps_per_episode = 100
learning_rate = 0.8
discount_factor = 0.95
exploration_prob = 0.2

# 初始化Q值表格，记录每个状态和动作的Q值
num_actions = env.action_space.n
num_states = env.observation_space.shape[0]
q_table = np.zeros((num_states, num_actions))

# 定义神经网络模型，用于近似Q值函数
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(num_states,)),
        tf.keras.layers.Dense(num_actions)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse')
    return model

# 训练Q-learning算法
def train_q_learning():
    model = create_model()

    for episode in range(num_episodes):
        state = env.reset()
        state = np.reshape(state, [1, num_states])

        for step in range(num_steps_per_episode):
            # 通过epsilon-greedy策略选择动作
            if random.uniform(0, 1) < exploration_prob:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1, num_states])

            # 更新Q值函数
            target = reward
            if not done:
                target = reward + discount_factor * np.max(q_table[next_state, :])
            q_table[state, action] = (1 - learning_rate) * q_table[state, action] + learning_rate * target

            state = next_state
            print(reward)
            if done:
                break

    return model

# 训练模型
trained_model = train_q_learning()

# 使用训练好的模型进行游戏
state = env.reset()
state = np.reshape(state, [1, num_states])

for step in range(num_steps_per_episode):
    action = np.argmax(trained_model.predict(state))
    next_state, _, done, _ = env.step(action)

    if done:
        break

    state = np.reshape(next_state, [1, num_states])

env.close()
'''
print('.')